In [1]:
# I am getting the code from: https://github.com/navreeetkaur/bayesian-network-learning
# Specifically baysnet.py,
# So, this notebook needs to be called from other notebooks,

In [2]:
from __future__ import division
#Wow, after 5 years, it's still from futures!! Damn

from collections import OrderedDict
import numpy as np 
import pandas as pd 
import time


In [3]:
class Graph_Node():
	"""Our graph consists of a list of nodes where 
    each node is represented as follows"""

	def __init__(self, name, n, vals):
		self.Node_Name = name # Variable name 
		self.nvalues = n # Number of categories a 
        #variable represented by this node can take
        
		self.values = vals # Categories of possible values
		self.Children =  [] # Children of a particular node 
        #- these are index of nodes in graph.
        
		self.Parents = [] # Parents of a particular node
        #- note these are names of parents
        
		self.CPT = []
		self.cpt_data =  pd.DataFrame() # conditional 
        # probability table as a DataFrame (counts) 
        
		self.markov_blanket = [] # List of nodes in 
        # the Markov Blanket - note that these are the
        # names of the nodes

	def get_name(self):
		return self.Node_Name

	def get_children(self):
		return self.Children

	def get_Parents(self):
		return self.Parents

	def get_n_parents(self):
		return len(self.Parents)

	def get_CPT(self):
		return self.CPT

	def get_nvalues(self):
		return self.nvalues

	def get_values(self):
		return self.values

	def set_CPT(self, new_CPT):
		del(self.CPT[:])
		self.CPT = new_CPT

	def set_counts(self, new_counts):
		del(self.counts[:])
		self.counts = new_counts

	def set_MB(self, new_mb):
		self.markov_blanket = new_mb

	def set_cpt_data(self, new_cpt_data):
		self.cpt_data.drop(columns = list(self.cpt_data.columns))
		self.cpt_data = new_cpt_data

	def set_Parents(self, Parent_Nodes):
		self.Parents = Parent_Nodes

	def add_child(self, new_child_index):
		if new_child_index in self.Children:
			return 0
		else:
			self.Children.append(new_child_index)
			return 1

	def print_node(self):
		print(self.Node_Name)
		print(self.values)
		print(self.Parents)
		print(self.CPT)
		print




In [15]:
class network():
    """
    The whole network represted as a dictionary of nodes
    Pres_Graph: 
        Ordered Dictionary - Keys: variable names, 
                            Values: Node Objects
    MB:
        Ordered Dictionary - Keys: variable names, 
                        Values: List of names of the nodes in the 
                        markob blanket of the key
    """

    def __init__(self, Pres_Graph = OrderedDict(), 
                 MB = OrderedDict()):
        self.Pres_Graph = Pres_Graph
        self.MB = MB

    def addNode(self, node):
        self.Pres_Graph[node.Node_Name] = node

    def netSize(self):
        return len(self.Pres_Graph)
        
    def get_index(self, val_name):
        try:
            return self.Pres_Graph[val_name]
        except:
            print("No node of the name: " + str(val_name))
            print("Pres_Graph: ", self.Pres_Graph.keys())
            return None

    def get_nth_node(self, n):
        return self.Pres_Graph.values()[n]

    def search_node(self, val_name):
        try:
            return self.Pres_Graph[val_name]
        except:
            print("Node NOT found")
            return None

    def get_parent_nodes(self, node):
        parent_nodes = []
        parents = node.get_Parents()
        for p in parents:
            parent_nodes.append(self.search_node(p))
        return parent_nodes

    def get_children(self, val_name):
        Children = self.Pres_Graph[val_name].Children
        c = []
        for n in Children:
            c.append(self.Pres_Graph.keys()[n])
        return c

    def set_mb(self):
        for vals in self.Pres_Graph.keys():
            self.MB[vals] = markov_blanket(self, vals)
            

    def normalise_cpt(self, X):
        l = [X] + self.Pres_Graph[X].Parents + ['counts', 'p']
        cpt = self.Pres_Graph[X].cpt_data
        nvals = self.Pres_Graph[X].nvalues
        cardinality = cpt.shape[0]
        no_grps = int(cardinality/nvals)
        list_dfs = []
        df = pd.DataFrame()
        i=0 
        for n in range(no_grps):
            curr_df = pd.DataFrame(cpt.iloc[i:i+nvals, :])
            curr_df['p'] = normalise_counts(curr_df['counts'])
            df = df.append(curr_df)
            i = i + nvals
        self.Pres_Graph[X].cpt_data = df[l]

In [36]:
""" Reading network from .bif format """
def read_network(bif_filepath):
    Alarm = network()
    find = 0

    with open(bif_filepath, 'r') as  myfile: 
        while True:
            line = myfile.readline()
            line = line.strip()

            if line == '':
                break

            tokens = line.split()
            first_word = tokens[0]


            if first_word == "variable":
                values = []
                name = tokens[1] # random varible name
                line_ = myfile.readline() # read next line
                line_ = line_.strip()
                tokens_ = line_.split()
                for i in range(3,len(tokens_)-1):
                    values.append(tokens_[i])
                new_node = Graph_Node(name = name, n = len(values), vals = values)
                Alarm.addNode(new_node)

            
            if first_word == "probability":
                vals = []
                temp = tokens[2]
                node = Alarm.search_node(temp)
                index = Alarm.get_index(temp)
                i = 3
                # setting parents
                while True:
                    if tokens[i]==")":
                        break
                    node_ = Alarm.search_node(tokens[i])
                    node_.add_child(index)
                    vals.append(tokens[i])
                    i = i + 1

                node.set_Parents(vals)

                line_ = myfile.readline().replace(","," ")
                #print("line_:", line_)
                tokens_ = line_.split()
                curr_CPT = []
                for i in range(1,len(tokens_)-1):
                    curr_CPT.append(float(tokens_[i]))

                node.set_CPT(curr_CPT)

    myfile.close()

    return Alarm

In [35]:
line = "2,3,4"
line = line.replace(","," ")
line

'2 3 4'

In [27]:
bPath = "/media/aknirala/c6ed7430-6a77-48e0-920c-777be00c9dc21/PhD/Thesis/GraphMod/bayesian-network-learning/data/"
gold_alarm = bPath + "gold_alarm.bif"

In [28]:
G = read_network(gold_alarm)

In [37]:
earthquake = "earthquake.bif"
G = read_network(earthquake)
type(G), G.Pres_Graph

Node NOT found


AttributeError: 'NoneType' object has no attribute 'add_child'

In [31]:
G.Pres_Graph

OrderedDict([('"Hypovolemia"', <__main__.Graph_Node at 0x7fb5b65a4390>),
             ('"StrokeVolume"', <__main__.Graph_Node at 0x7fb5b65a4c88>),
             ('"LVFailure"', <__main__.Graph_Node at 0x7fb5b65a4400>),
             ('"LVEDVolume"', <__main__.Graph_Node at 0x7fb5b65a45f8>),
             ('"PCWP"', <__main__.Graph_Node at 0x7fb5b6586898>),
             ('"CVP"', <__main__.Graph_Node at 0x7fb5b65a4e10>),
             ('"History"', <__main__.Graph_Node at 0x7fb5b65a4dd8>),
             ('"MinVolSet"', <__main__.Graph_Node at 0x7fb5b65a4160>),
             ('"VentMach"', <__main__.Graph_Node at 0x7fb5b6586b70>),
             ('"Disconnect"', <__main__.Graph_Node at 0x7fb5b65ed278>),
             ('"VentTube"', <__main__.Graph_Node at 0x7fb5b65ed550>),
             ('"KinkedTube"', <__main__.Graph_Node at 0x7fb5b65ed780>),
             ('"Press"', <__main__.Graph_Node at 0x7fb5b65ed860>),
             ('"ErrLowOutput"', <__main__.Graph_Node at 0x7fb5b65ed8d0>),
             (